## Model training for 10 percent data variation
1. Batch size = 64
2. Neurons = 128


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
# from google.colab import drive
# drive.mount('/content/gdrive')
import os

In [14]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May7rd10percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [15]:
df.describe()

,h,v,s,omega,gamma,m,theta,Thrust,beta,time
count,221089.000000,221089.000000,2.210890e+05,221089.000000,221089.000000,221089.000000,221089.000000,221089.000000,221089.000000,221089.000000
mean,923.478003,102.870601,6.908459e+02,-0.016870,-1.484311,23153.406167,-1.481758,482935.884011,0.001837,21.706884
std,1334.673350,90.152141,2.535273e+02,0.127985,0.124744,1339.376075,0.353444,101507.477577,0.046149,9.133048
min,-0.059782,4.996700,-5.031700e-07,-0.431760,-1.709600,21357.000000,-2.805200,356960.000000,-0.208800,0.000000
25%,11.201000,18.925000,5.572900e+02,-0.072434,-1.571100,22054.000000,-1.570800,403840.000000,-0.013007,15.873000
50%,252.570000,80.095000,8.201600e+02,-0.006814,-1.552900,22775.000000,-1.558000,442150.000000,-0.002892,24.555000
75%,1336.200000,174.840000,8.611500e+02,-0.000024,-1.424600,24081.000000,-1.308800,542160.000000,0.001759,29.015000
max,5263.200000,342.110000,9.080300e+02,0.436880,-0.936550,26230.000000,-0.325280,769190.000000,0.210660,34.102000


In [16]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [17]:
i = 7;
model = create_model2(2**i)
# model.load_weights(f"/content/gdrive/My Drive/Last data /1percent model/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [18]:
model.predict(X_test)


array([[0.5000009 , 0.4999882 ],
       [0.5000028 , 0.49998745],
       [0.50000083, 0.49999076],
       ...,
       [0.5000019 , 0.49998888],
       [0.5000026 , 0.49998564],
       [0.49999967, 0.49998665]], dtype=float32)

In [19]:
y_test

array([[0.77228732, 0.49065417],
       [0.33520122, 0.49494994],
       [0.18111249, 0.60267487],
       ...,
       [0.93282876, 0.464738  ],
       [0.068651  , 0.63641587],
       [0.08703879, 0.31978496]])

In [21]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss = tf.keras.losses.mean_squared_error
def lr_sch3(epoch,lr):
    if (epoch >0) &(epoch<300):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>300):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch3,verbose = 1)
bs = 64;
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')

The Number of Hidden units used is:  128


5906/5906 [==============================] - 11s 2ms/step - loss: 0.0552 - accuracy: 0.2254


[0.0551857128739357, 0.22543655335903168]

In [ ]:
model.load_weightsweights(f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [23]:
hist = model.fit(X_train,y_train,epochs = 100,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/1000
2953/2953 [==============================] - 13s 4ms/step - loss: 0.0105 - accuracy: 0.8715 - val_loss: 0.0069 - val_accuracy: 0.8894 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0064 - accuracy: 0.8922 - val_loss: 0.0064 - val_accuracy: 0.8961 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0057 - accuracy: 0.8955 - val_loss: 0.0051 - val_accuracy: 0.8974 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0052 - accuracy: 0.8985 - val_loss: 0.0054 - val_accuracy: 0.9001 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/1000
2953/2953 [================

Epoch 36/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0022 - accuracy: 0.9376 - val_loss: 0.0019 - val_accuracy: 0.9498 - lr: 0.0010

Epoch 37: LearningRateScheduler setting learning rate to 0.001.
Epoch 37/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0022 - accuracy: 0.9377 - val_loss: 0.0016 - val_accuracy: 0.9588 - lr: 0.0010

Epoch 38: LearningRateScheduler setting learning rate to 0.001.
Epoch 38/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0022 - accuracy: 0.9376 - val_loss: 0.0018 - val_accuracy: 0.9467 - lr: 0.0010

Epoch 39: LearningRateScheduler setting learning rate to 0.001.
Epoch 39/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0022 - accuracy: 0.9376 - val_loss: 0.0018 - val_accuracy: 0.9435 - lr: 0.0010

Epoch 40: LearningRateScheduler setting learning rate to 0.001.
Epoch 40/1000
2937/2953 [============================>.] - ETA: 0s - loss: 0.0021 - accuracy: 0.

2953/2953 [==============================] - 12s 4ms/step - loss: 0.0013 - accuracy: 0.9592 - val_loss: 0.0010 - val_accuracy: 0.9706 - lr: 6.3760e-04

Epoch 70: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 70/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0013 - accuracy: 0.9594 - val_loss: 0.0014 - val_accuracy: 0.9529 - lr: 6.3760e-04

Epoch 71: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 71/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0013 - accuracy: 0.9575 - val_loss: 0.0011 - val_accuracy: 0.9626 - lr: 6.3760e-04

Epoch 72: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 72/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 0.0013 - accuracy: 0.9584 - val_loss: 0.0012 - val_accuracy: 0.9570 - lr: 6.3760e-04

Epoch 73: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 73/1000
2953/2953 [==============================] - 12s 4ms/step - loss


Epoch 102: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 102/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 9.1047e-04 - accuracy: 0.9683 - val_loss: 7.5645e-04 - val_accuracy: 0.9756 - lr: 4.0660e-04

Epoch 103: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 103/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 9.0078e-04 - accuracy: 0.9688 - val_loss: 0.0012 - val_accuracy: 0.9580 - lr: 4.0660e-04

Epoch 104: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 104/1000
2953/2953 [==============================] - 12s 4ms/step - loss: 8.6355e-04 - accuracy: 0.9696 - val_loss: 8.1855e-04 - val_accuracy: 0.9743 - lr: 4.0660e-04

Epoch 105: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 105/1000
1804/2953 [=================>............] - ETA: 4s - loss: 8.3529e-04 - accuracy: 0.9699

KeyboardInterrupt: 

In [24]:
def lr_sch100(epoch,lr):
    epoch = epoch+100
    if (epoch >0) &(epoch<125):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>125):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
lr_scheduler2 = tf.keras.callbacks.LearningRateScheduler(lr_sch100,verbose = 1)
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))

In [25]:
model.load_weights(f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [ ]:
hist = model.fit(X_train,y_train,epochs = 600,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler2,cp_callback2],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 1/600
2953/2953 [==============================] - 13s 4ms/step - loss: 9.1224e-04 - accuracy: 0.9687 - val_loss: 8.6851e-04 - val_accuracy: 0.9729 - lr: 4.0660e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 2/600
2953/2953 [==============================] - 13s 4ms/step - loss: 9.1803e-04 - accuracy: 0.9680 - val_loss: 6.3282e-04 - val_accuracy: 0.9774 - lr: 4.0660e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 3/600
2953/2953 [==============================] - 12s 4ms/step - loss: 9.1557e-04 - accuracy: 0.9681 - val_loss: 7.9038e-04 - val_accuracy: 0.9718 - lr: 4.0660e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 4/600
2953/2953 [==============================] - 12s 4ms/step - loss: 8.9270e-04 - accuracy: 0.9687 - val_loss: 8.3350e-04 - val_accuracy: 0.9755 - lr: 4.0660e-04

Epoch 5: LearningRateScheduler setting lear

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2953/2953 [==============================] - 13s 4ms/step - loss: 7.2991e-04 - accuracy: 0.9735 - val_loss: 9.4370e-04 - val_accuracy: 0.9666 - lr: 3.4561e-04

Epoch 47: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 47/600
2953/2953 [==============================] - 13s 4ms/step - loss: 7.4224e-04 - accuracy: 0.9726 - val_loss: 6.4031e-04 - val_accuracy: 0.9732 - lr: 3.4561e-04

Epoch 48: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 48/600
2953/2953 [==============================] - 12s 4ms/step - loss: 7.2179e-04 - accuracy: 0.9730 - val_loss: 5.1998e-04 - val_accuracy: 0.9804 - lr: 3.4561e-04

Epoch 49: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 49/600
2953/2953 [==============================] - 13s 4ms/step - loss: 7.4854e-04 - accuracy: 0.9722 - val_loss: 0.0012 - val_accuracy: 0.9630 - lr: 3.4561e-04

Epoch 50: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 50/600
2953/2953 [=========================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2953/2953 [==============================] - 13s 4ms/step - loss: 7.1962e-04 - accuracy: 0.9732 - val_loss: 5.2912e-04 - val_accuracy: 0.9802 - lr: 3.4561e-04

Epoch 59: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 59/600
2953/2953 [==============================] - 13s 4ms/step - loss: 6.9898e-04 - accuracy: 0.9738 - val_loss: 5.6690e-04 - val_accuracy: 0.9769 - lr: 3.4561e-04

Epoch 60: LearningRateScheduler setting learning rate to 0.00034561.
Epoch 60/600
2929/2953 [============================>.] - ETA: 0s - loss: 6.8842e-04 - accuracy: 0.9743
Epoch 60: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
2953/2953 [==============================] - 13s 4ms/step - loss: 6.8730e-04 - accuracy: 0.9743 - val_loss: 7.7900e-04 - val_accuracy: 0.9729 - lr: 3.4561e-04

Epoch 61: LearningRateScheduler setting learning rate to 0.000293768.
Epoch 61/600
2953/2953 [=

...

2953/2953 [==============================] - 13s 4ms/step - loss: 2.5002e-04 - accuracy: 0.9887 - val_loss: 2.5020e-04 - val_accuracy: 0.9909 - lr: 2.5662e-05

Epoch 379: LearningRateScheduler setting learning rate to 2.5662e-05.
Epoch 379/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.5178e-04 - accuracy: 0.9885 - val_loss: 2.4813e-04 - val_accuracy: 0.9906 - lr: 2.5662e-05

Epoch 380: LearningRateScheduler setting learning rate to 2.5662e-05.
Epoch 380/600
2874/2953 [============================>.] - ETA: 0s - loss: 2.4994e-04 - accuracy: 0.9889
Epoch 380: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
2953/2953 [==============================] - 13s 4ms/step - loss: 2.5149e-04 - accuracy: 0.9889 - val_loss: 4.6132e-04 - val_accuracy: 0.9806 - lr: 2.5662e-05

Epoch 381: LearningRateScheduler setting learning rate to 2.1813e-05.
Epoch 381/600
2953/2

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2953/2953 [==============================] - 13s 4ms/step - loss: 2.4879e-04 - accuracy: 0.9885 - val_loss: 2.5352e-04 - val_accuracy: 0.9907 - lr: 2.1813e-05

Epoch 398: LearningRateScheduler setting learning rate to 2.1813e-05.
Epoch 398/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.4435e-04 - accuracy: 0.9889 - val_loss: 3.0092e-04 - val_accuracy: 0.9875 - lr: 2.1813e-05

Epoch 399: LearningRateScheduler setting learning rate to 2.1813e-05.
Epoch 399/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.4566e-04 - accuracy: 0.9889 - val_loss: 2.5496e-04 - val_accuracy: 0.9898 - lr: 2.1813e-05

Epoch 400: LearningRateScheduler setting learning rate to 2.1813e-05.
Epoch 400/600
2861/2953 [============================>.] - ETA: 0s - loss: 2.4270e-04 - accuracy: 0.9889
Epoch 400: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/10 percent variation/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
2953/2


Epoch 429: LearningRateScheduler setting learning rate to 1.576e-05.
Epoch 429/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.3348e-04 - accuracy: 0.9894 - val_loss: 2.5768e-04 - val_accuracy: 0.9897 - lr: 1.5760e-05

Epoch 430: LearningRateScheduler setting learning rate to 1.576e-05.
Epoch 430/600
2953/2953 [==============================] - 12s 4ms/step - loss: 2.3832e-04 - accuracy: 0.9893 - val_loss: 2.4674e-04 - val_accuracy: 0.9906 - lr: 1.5760e-05

Epoch 431: LearningRateScheduler setting learning rate to 1.576e-05.
Epoch 431/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.3497e-04 - accuracy: 0.9892 - val_loss: 2.6537e-04 - val_accuracy: 0.9900 - lr: 1.5760e-05

Epoch 432: LearningRateScheduler setting learning rate to 1.576e-05.
Epoch 432/600
2953/2953 [==============================] - 12s 4ms/step - loss: 2.3472e-04 - accuracy: 0.9893 - val_loss: 2.5902e-04 - val_accuracy: 0.9906 - lr: 1.5760e-05

Epoch 433: LearningRateSche

2953/2953 [==============================] - 13s 4ms/step - loss: 2.3054e-04 - accuracy: 0.9896 - val_loss: 2.6210e-04 - val_accuracy: 0.9891 - lr: 1.3396e-05

Epoch 461: LearningRateScheduler setting learning rate to 1.1387e-05.
Epoch 461/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.2797e-04 - accuracy: 0.9895 - val_loss: 2.6567e-04 - val_accuracy: 0.9889 - lr: 1.1387e-05

Epoch 462: LearningRateScheduler setting learning rate to 1.1387e-05.
Epoch 462/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.2889e-04 - accuracy: 0.9895 - val_loss: 2.6096e-04 - val_accuracy: 0.9898 - lr: 1.1387e-05

Epoch 463: LearningRateScheduler setting learning rate to 1.1387e-05.
Epoch 463/600
2953/2953 [==============================] - 13s 4ms/step - loss: 2.2957e-04 - accuracy: 0.9895 - val_loss: 2.4153e-04 - val_accuracy: 0.9913 - lr: 1.1387e-05

Epoch 464: LearningRateScheduler setting learning rate to 1.1387e-05.
Epoch 464/600
2953/2953 [=============

model.save(f"/content/gdrive/My Drive/Last data /10percent model/best models/7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt")

In [ ]:
lr =hist.history.get('lr')[-1] # It takes the last used learning rate.
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.mean_squared_error
def ModelsWith99PercentAccuracy(epoch,lr):
    if(epoch>0):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
      return round(lr,9)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(ModelsWith99PercentAccuracy,verbose = 1)


In [ ]:
hist = model.fit(X_train,y_train,epochs = 20,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))

## Antinormalizing all the input and output features


In [ ]:
def AntiNorm(Norm_value,actual):
  return (max(actual)-min(actual))*Norm_value+min(actual)

In [ ]:
h_norm = X_norm[:,0]
v_norm = X_norm[:,1]
s_norm = X_norm[:,2]
omega_norm = X_norm[:,3]
gamma_norm = X_norm[:,4]
m_norm = X_norm[:,5]
theta_norm = X_norm[:,6]
Thrust_norm = y_norm[:,0]
beta_norm = y_norm[:,1]
h = X[:,0]
v = X[:,1]
s = X[:,2]
omega = X[:,3]
gamma = X[:,4]
m = X[:,5]
theta = X[:,6]
Thrust = y[:,0]
beta = y[:,1]

In [ ]:
# predicted outputs
y_pred = model.predict(X_norm)
Thrust_p = AntiNorm(y_pred[:,0],Thrust)
beta_p = AntiNorm(y_pred[:,1],beta)

In [ ]:
plt.plot(beta_p[0:3030])
plt.plot(beta[0:3030])
plt.legend(["Predicted","Actual"])
plt.xlabel("Number of points")
plt.ylabel("Nozzle Gamble")

plt.plot(Thrust_p[0:3030])
plt.plot(Thrust[0:3030])
plt.legend(["Predicted","Actual"])
plt.xlabel("Number of points")
plt.ylabel("Thrust")